## Задание

- Взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes, обучить модель GPT для генерации своих цитат

- Взять новостные данные из https://github.com/natasha/corus load_lenta2, нам понадобиться сам текст и заголовок, обучить модель T5/ или GPT для генерации заголовков для статей


---

In [ ]:
#!wget https://www.dropbox.com/s/ykqk49a8avlmnaf/ru_all_split.tar.gz

In [ ]:
import os

os.listdir('./ru_all_split')

In [ ]:
# Импорт данных в pandas dataframe

import pandas as pd

train_data = pd.read_json('./ru_all_split/ru_all_train.jsonl', lines=True) 
test_data = pd.read_json('./ru_all_split/ru_all_test.jsonl', lines=True)

In [ ]:
# Импорт данных в датасет hugging face

from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
# Ограничим размер датасета (при исходном умирает ядро)

train_dataset = train_dataset.select(range(100))
test_dataset = test_dataset.select(range(50))

In [ ]:
train_dataset['title'][0]

---

In [ ]:
# Модель

model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [ ]:
def len_tok(text):
    return len(text.split())

In [ ]:
# Максимальная длинна текста и заголовка

max_len_text, max_len_tl = max(map(len_tok, train_dataset['text'])), max(map(len_tok, train_dataset['title']))
max_len_text, max_len_tl

In [ ]:
max(train_dataset['title'])

In [ ]:
# Токенизация

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_len_text)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=8)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=8)

train_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# Сохранение на диск

train_dataset.save_to_disk('./ru_all_split/train')
test_dataset.save_to_disk('./ru_all_split/test')

In [ ]:
# Загрузка модели

from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# Тренировочные аргументы

output_dir = './ru_all_split/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.000001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_headlines', # Wandb run name
    logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

In [ ]:
# Обучение

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

In [ ]:
# Сохранение модели

trainer.save_model(output_dir + '/model')

In [ ]:
# Индекс для создания заголовка

index = 11

In [ ]:
# Работа модели

import torch

input_text = test_dataset['text'][index]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids']
    source_mask = tokenized_text['attention_mask'] 

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
# Сравнение исходного и предсказанного заголовка

print('\nИсходный заголовок:\n{}'.format(test_dataset['title'][index]))
print('\nПредсказанный заголовок:\n{}'.format(pred))